In [6]:
%run metricas.ipynb 

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/tomy/.pyenv/versions/py3.6/bin/python (found version "3.6.9") 
-- Found PythonLibs: /home/tomy/.pyenv/versions/3.6.9/lib/libpython3.6m.a
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/tomy/Desktop/tp3/met-tp3/metnum-tp3-2020-1c
-- C

In [7]:
def segXcatNum(df, n , palabra):
    #la categoria seleccionada tiene que contener numeros, no palabras
    #n = cantidad de particiones
    #data es la matriz con toda la informacion
    #palabra es la columna por la cual segmentaremos
    
            
    df = df[df['ciudad']=='Mérida'].dropna() #TODO sin esto no anda, averiguar por que
    
    tam = df.shape[0]
    linear_regressor = metnum.LinearRegression()
    
    #TODO no supe como ordenarlo con numpy, ya que pierdo el titulo cuando paso de pd a np.
    #aclaro xq no se cuanto mas tarda ordenar pandas o numpy, y no se si cambia mucho o no
    df.sort_values("precio") #ordena de menor a mayor
    
    #tomo las columnas de PALABRA y los precios
    categoria = df[palabra].values
    precios = df['precio'].values
    vector = [] #esto es un asco
    for i in range(n):
            
        #tomo n bloques de longitud tam/n
        bloqueCategoria = categoria[int(i * tam/n) : int((i+1) * (tam/n))]
        bloquePrecios = precios[int(i * tam/n) : int((i+1) * tam/n)]
       
    #reshapeo
        bloqueCategoria = bloqueCategoria.reshape(len(bloqueCategoria),1)
        bloquePrecios = bloquePrecios.reshape(len(bloquePrecios),1)

        
        #fit y prediccion, que voy guardando en el conjunto ordenado
        linear_regressor.fit(bloqueCategoria, bloquePrecios)
        lin = linear_regressor.predict(bloqueCategoria)
        
        #reshapeo para poder meterlo en un array con el formato necesario
        lin = lin.reshape(lin.shape[0])
        vector.extend(lin)
    
    df['prediction'] = vector 
    return vector


In [9]:
segmentado = segXcatNum(df, 5, 'metroscubiertos')
df = df[df['ciudad']=='Zapopan'].dropna() #TODO sin esto no anda, averiguar por que
coefCorrel(segmentado, df['precio'])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 3648